In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master')

Mounted at /content/gdrive


In [2]:
import json
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [3]:
!wget -P . https://early-access-program.debater.res.ibm.com/sdk/python_api.tar.gz
!tar -xvf python_api.tar.gz
!cd python_api ; pip install .
!rm -f python_api.tar.gz*

--2022-05-23 23:38:46--  https://early-access-program.debater.res.ibm.com/sdk/python_api.tar.gz
Resolving early-access-program.debater.res.ibm.com (early-access-program.debater.res.ibm.com)... 169.55.93.82
Connecting to early-access-program.debater.res.ibm.com (early-access-program.debater.res.ibm.com)|169.55.93.82|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 442592 (432K) [application/x-gzip]
Saving to: ‘./python_api.tar.gz’

python_api.tar.gz   100%[===================>] 432.22K  2.27MB/s    in 0.2s    

2022-05-23 23:38:47 (2.27 MB/s) - ‘./python_api.tar.gz’ saved [442592/442592]

python_api/
python_api/__init__.py
python_api/setup.py
python_api/README.md
python_api/debater_python_api/
python_api/debater_python_api/__init__.py
python_api/debater_python_api/examples/
python_api/debater_python_api/examples/__init__.py
python_api/debater_python_api/examples/evidence_detection_example.py
python_api/debater_python_api/examples/term_wikifier_example.py
python_ap

In [4]:
# (C) Copyright IBM Corp. 2020.
import os
from debater_python_api.api.clients.narrative_generation_client import Polarity
from debater_python_api.api.debater_api import DebaterApi
from debater_python_api.api.sentence_level_index.client.sentence_query_base import SimpleQuery
from debater_python_api.api.sentence_level_index.client.sentence_query_request import SentenceQueryRequest

In [5]:
from spacy.lang.en import English
sent_split = English()
sent_split.add_pipe(sent_split.create_pipe('sentencizer'))
# sentencizer allow custom sentence boundary detection logic that doesn’t require the dependency parse
nlp = spacy.load('en_core_web_sm')

In [6]:
train_args= pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/KPA_2021_shared_task/kpm_data/arguments_train.csv')
valid_args = pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/KPA_2021_shared_task/kpm_data/arguments_dev.csv')
test_args = pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/KPA_2021_shared_task/test_data/arguments_test.csv')

In [7]:
train_kps= pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/KPA_2021_shared_task/kpm_data/key_points_train.csv')
valid_kps = pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/KPA_2021_shared_task/kpm_data/key_points_dev.csv')
test_kps = pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/KPA_2021_shared_task/test_data/key_points_test.csv')

In [8]:
training_df = pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/data/training_df.csv')
valid_df = pd.read_csv('/content/gdrive/MyDrive/collab/NLP/argmining-21-keypoint-analysis-sharedtask-code-master/data/valid_df.csv')

In [9]:
valid_args['sents'] = valid_args.argument.apply(lambda x: [x.text for x in sent_split(x).sents if list(nlp(x.text))[0].pos_ != 'PRON'])
train_args['sents'] = train_args.argument.apply(lambda x: [x.text for x in sent_split(x).sents if list(nlp(x.text))[0].pos_ != 'PRON'])
test_args['sents'] = test_args.argument.apply(lambda x: [x.text for x in sent_split(x).sents if list(nlp(x.text))[0].pos_ != 'PRON'])

In [10]:
debater_api = DebaterApi('0abeffa5335cc942fc7c43e75d41fe33L05')
arg_quality_client = debater_api.get_argument_quality_client()

def score_candidates(topic, sentences):
    sentence_topic_dicts = [{'sentence' : sentence, 'topic' : topic } for sentence in sentences]
    scores = arg_quality_client.run(sentence_topic_dicts)
    return list(zip(sentences, scores))

In [ ]:
test_args['sents_with_scores'] = test_args.apply(lambda row: score_candidates(row['topic'], row['sents']) , axis=1)

ArgumentQualityClient: 100%|██████████| 1/1 [00:06<00:00,  6.01s/it]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 100%|██████████| 1/1 [00:06<00:00,  6.01s/it]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 100%|██████████| 2/2 [00:06<00:00,  3.01s/it]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 100%|██████████| 1/1 [00:05<00:00,  5.94s/it]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 100%|██████████| 1/1 [00:05<00:00,  5.94s/it]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0it [00:00, ?it/s]
ArgumentQualityClient: 0i

In [ ]:
valid_args['sents_with_scores'] = valid_args.apply(lambda row: score_candidates(row['topic'], row['sents']) , axis=1)

In [ ]:
valid_args.head()

In [ ]:
train_args['sents_with_scores'] = train_args.apply(lambda row: score_candidates(row['topic'], row['sents']) , axis=1)

In [ ]:
train_args.to_pickle('../data/train_args_with_scores.pkl')
valid_args.to_pickle('../data/valid_args_with_scores.pkl')

In [ ]:
test_args.to_pickle('../data/test_args_with_scores.pkl')

In [ ]:
candidate_kps = [cand for cands in valid_args.sents_with_scores.tolist() for cand in cands if cand[1] > 0.5 and cand]

In [ ]:
candidate_kps[0:3]

In [ ]:
len(candidate_kps)

### Generate labels for sentences:

In [ ]:
import spacy 

In [ ]:
train_args = pd.read_pickle('../data/train_args_with_scores.pkl')
valid_args = pd.read_pickle('../data/valid_args_with_scores.pkl')

In [ ]:
def find_matching_sentence(sents, kp):
    if len(sents) == 0:
        return []

    def overlap(t1, t2):
        tokens1 = [token.text for token in nlp(t1) if not token.is_stop]
        tokens2 = [token.text for token in nlp(t2) if not token.is_stop]

        return len(set(tokens1).intersection(set(tokens2)))/len(set(tokens1).union(set(tokens2)))
    
    overlaps = [overlap(sent, kp) for sent in sents]
    return sents[np.argmax(overlaps)]

Score topic keypoints and add them as extra sentences into the corrsponding arguments:

In [ ]:
train_topic_kps = train_kps.groupby(['topic', 'stance']).agg({
    'key_point': lambda x: list(x)
}).reset_index()

In [ ]:
train_topic_kps['scored_kps'] = train_topic_kps.apply(lambda row: score_candidates(row['topic'], row['key_point']) , axis=1)

In [ ]:
train_topic_kps.head()

In [ ]:
train_topic_kps.to_pickle('../data/train_topic_keypoints_with_scores.pkl')

In [ ]:
arg_kps_df = training_df[training_df.label == 1].groupby('arg_id').agg({
    'key_point': lambda x: list(x)
}).reset_index()
arg_kps_dict = pd.Series(arg_kps_df.key_point.values, index=arg_kps_df.arg_id).to_dict()

In [ ]:
train_args['gt-kps'] = train_args.apply(lambda row: arg_kps_dict[row['arg_id']] if row['arg_id'] in arg_kps_dict else [], axis=1)

In [ ]:
train_args['gt-cand-kps'] = train_args.apply(lambda row: [find_matching_sentence(row['sents'], kp) for kp in arg_kps_dict[row['arg_id']]] if row['arg_id'] in arg_kps_dict else [] , axis=1)

In [ ]:
train_args.to_pickle('../data/train_args_with_scores.pkl')
train_args.to_pickle('../data/train_args_with_scores.csv')